## Imports

In [4]:
pip install ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 480.1 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 670.6 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
import pandas as pd
import re
from ftfy import fix_text
import unidecode
import json

In [9]:
df = pd.read_json("gg2013.json")

# Extract user information into separate columns
df['user_screen_name'] = df['user'].apply(lambda x: x['screen_name'])
df['user_id'] = df['user'].apply(lambda x: x['id'])

# Drop the original 'user' column as we've extracted the needed information
df = df.drop('user', axis=1)

# Convert timestamp_ms to datetime
df['timestamp'] = pd.to_datetime(df['timestamp_ms'], unit='ms')

# Drop the original timestamp_ms column
df = df.drop('timestamp_ms', axis=1)

# Reorder columns for better readability
df = df[['id', 'timestamp', 'user_id', 'user_screen_name', 'text']]

# Display the first few rows of the DataFrame
print(df.head())

                   id           timestamp    user_id user_screen_name  \
0  290620657987887104 2013-01-14 00:45:38  557374298        Dozaaa_xo   
1  290620657887219713 2013-01-14 00:45:38   14648726     theAmberShow   
2  290620657828524032 2013-01-14 00:45:38   35498686         SweetyPW   
3  290620657799159809 2013-01-14 00:45:38  144430208   _NicoleEdwards   
4  290620657778188288 2013-01-14 00:45:38  134953223    lolaogunnaike   

                                                text  
0             JLo's dress! #eredcarpet #GoldenGlobes  
1  What's making Sofia Vergara's boobs stay like ...  
2  RT @FabSugar: Kerry Washington is EVERYTHING. ...  
3     Anne Hathaway has got me living. #GoldenGlobes  
4  Jennifer Lopez's lace dress? Thoughts? #Golden...  


In [10]:
def preprocess_text(text):
    # Fix encoding issues (ampersands, etc.) using ftfy
    text = fix_text(text)
    
    # Remove non-ASCII characters (emojis, unicode symbols) using unidecode
    text = unidecode.unidecode(text)
    
    # Remove extra whitespace, tabs, and newlines (substitute with single spaces)
    # If we want to keep tabs/newline characters: text = re.sub(' +', ' ', text)
    text = " ".join(text.split())
    
    return text

# Apply preprocessing
df['clean_text'] = df['text'].apply(preprocess_text)

# Display new text
df['clean_text'].head()

def preprocess_text(text):
    # Fix encoding issues (ampersands, etc.) using ftfy
    text = fix_text(text)
    
    # Remove non-ASCII characters (emojis, unicode symbols) using unidecode
    text = unidecode.unidecode(text)
    
    # Extract hashtags and links
    hashtags = re.findall(r'#\w+', text)  # Extract hashtags
    links = re.findall(r'http[s]?://\S+', text)  # Extract URLs
    
    # Remove hashtags and links from the original text
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'http[s]?://\S+', '', text)  # Remove URLs
    
    # Remove extra whitespace, tabs, and newlines (substitute with single spaces)
    # If we want to keep tabs/newline characters: text = re.sub(' +', ' ', text)
    text = " ".join(text.split())
    
    return text, hashtags, links

# Apply preprocessing
df[['cleaned_text', 'hashtags', 'links']] = df['text'].apply(
    lambda x: pd.Series(preprocess_text(x))
)

# Display the updated DataFrame
df[['text', 'cleaned_text', 'hashtags', 'links']].head()

,text,cleaned_text,hashtags,links
0,JLo's dress! #eredcarpet #GoldenGlobes,JLo's dress!,"[#eredcarpet, #GoldenGlobes]",[]
1,What's making Sofia Vergara's boobs stay like ...,What's making Sofia Vergara's boobs stay like ...,[#GoldenGlobes],[]
2,RT @FabSugar: Kerry Washington is EVERYTHING. ...,RT @FabSugar: Kerry Washington is EVERYTHING. ...,[#goldenglobes],[http://t.co/3feH7MvO]
3,Anne Hathaway has got me living. #GoldenGlobes,Anne Hathaway has got me living.,[#GoldenGlobes],[]
4,Jennifer Lopez's lace dress? Thoughts? #Golden...,Jennifer Lopez's lace dress? Thoughts?,[#GoldenGlobes],[]


## Output

In [7]:
# Sample inputs
host = "Seth Meyers"
awards = {
    "Best Motion Picture - Drama": {
        "Presenters": ["Barbara Streisand"],
        "Nominees": [
            "Three Billboards Outside Ebbing, Missouri", 
            "Call Me by Your Name", 
            "Dunkirk", 
            "The Post", 
            "The Shape of Water"
        ],
        "Winner": "Three Billboards Outside Ebbing, Missouri"
    },
    "Best Motion Picture - Musical or Comedy": {
        "Presenters": ["Alicia Vikander", "Michael Keaton"],
        "Nominees": [
            "Lady Bird", 
            "The Disaster Artist", 
            "Get Out", 
            "The Greatest Showman", 
            "I, Tonya"
        ],
        "Winner": "Lady Bird"
    }
}

def format_human_readable(host, awards):
    result = f"Host: {host}\n\n"
    for award_name, details in awards.items():
        result += f"Award: {award_name}\n"
        result += f"Presenters: {', '.join(details['Presenters'])}\n"
        result += f'''Nominees: {', '.join([f'"{nominee}"' for nominee in details['Nominees']])}\n'''
        result += f"Winner: \"{details['Winner']}\"\n\n"
    return result

def format_json(host, awards):
    json_result = {
        "Host": host
    }
    json_result.update(awards)
    return json.dumps(json_result, indent=4)


human_readable_output = format_human_readable(host, awards)
json_output = format_json(host, awards)

print("Human-Readable Format:\n")
print(human_readable_output)

print("JSON Format:\n")
print(json_output)

Human-Readable Format:

Host: Seth Meyers

Award: Best Motion Picture - Drama
Presenters: Barbara Streisand
Nominees: "Three Billboards Outside Ebbing, Missouri", "Call Me by Your Name", "Dunkirk", "The Post", "The Shape of Water"
Winner: "Three Billboards Outside Ebbing, Missouri"

Award: Best Motion Picture - Musical or Comedy
Presenters: Alicia Vikander, Michael Keaton
Nominees: "Lady Bird", "The Disaster Artist", "Get Out", "The Greatest Showman", "I, Tonya"
Winner: "Lady Bird"


JSON Format:

{
    "Host": "Seth Meyers",
    "Best Motion Picture - Drama": {
        "Presenters": [
            "Barbara Streisand"
        ],
        "Nominees": [
            "Three Billboards Outside Ebbing, Missouri",
            "Call Me by Your Name",
            "Dunkirk",
            "The Post",
            "The Shape of Water"
        ],
        "Winner": "Three Billboards Outside Ebbing, Missouri"
    },
    "Best Motion Picture - Musical or Comedy": {
        "Presenters": [
            "Ali